In [46]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import scipy as sp
path='/home/student303/oyx/almama/'

In [47]:
train = pd.read_hdf(path+ 'somefeature/two_column_train_feature.h5', key = 'a')
test = pd.read_hdf(path+ 'somefeature/two_column_test_feature.h5', key = 'a')
#y_train = pd.read_csv(path + 'somefeature/y_train.csv')

In [53]:
train.head()

,context_id,context_page_id,context_timestamp,day,hour,instance_id,is_trade,item_brand_id,item_category_hash,item_category_list,...,user_age_level->user_gender_id_cvr,user_age_level->user_id_cvr,user_age_level->user_occupation_id_cvr,user_age_level->user_star_level_cvr,user_gender_id->user_id_cvr,user_gender_id->user_occupation_id_cvr,user_gender_id->user_star_level_cvr,user_id->user_occupation_id_cvr,user_id->user_star_level_cvr,user_occupation_id->user_star_level_cvr
0,5035994110420503338,4002,1537465939,21,1,4293715837363757697,0.0,1975590437749032870,5332,7908382889764677758;5799347067982556520,...,0.006536,NaN,0.012579,0.016794,NaN,0.018857,0.020409,NaN,NaN,0.020785
1,7972645707776427108,4015,1537537076,21,21,70764105903083828,0.0,9057103201734987852,9521,7908382889764677758;8277336076276184272,...,0.019138,NaN,0.018210,0.019691,NaN,0.018857,0.020409,NaN,NaN,0.020785
2,6797298244965748241,4002,1537539269,21,22,3954908518456917085,0.0,5520678735822176314,1443,7908382889764677758;5755694407684602296,...,0.019138,NaN,0.018210,0.019691,NaN,0.018857,0.020409,NaN,NaN,0.020785
3,7455406367907195,4001,1537509234,21,13,4521314384240415059,0.0,9057103201734987852,9521,7908382889764677758;8277336076276184272,...,0.019138,NaN,0.018210,0.018445,NaN,0.018857,0.018685,NaN,NaN,0.018967
4,1422823996606455864,4001,1537494715,21,9,6005676298575614673,0.0,5520678735822176314,1443,7908382889764677758;5755694407684602296,...,0.023138,NaN,0.027476,0.026915,NaN,0.018857,0.018722,NaN,NaN,0.020053


In [48]:
droplist = ['item_category_list',
            'item_property_list',
            'user_id',
            'context_id',
           #'context_page_id',
            'predict_category_property',
            #'shop_id'
           ]
df_test = test.drop(droplist, axis=1)
df_train = train.drop(droplist, axis=1)
# df_train.drop('is_trade', axis=1, inplace = True)
combine = [df_train, df_test]

In [49]:
df_train.columns

Index(['context_page_id', 'context_timestamp', 'day', 'hour', 'instance_id',
       'is_trade', 'item_brand_id', 'item_category_hash', 'item_city_id',
       'item_collected_level',
       ...
       'user_age_level->user_gender_id_cvr', 'user_age_level->user_id_cvr',
       'user_age_level->user_occupation_id_cvr',
       'user_age_level->user_star_level_cvr', 'user_gender_id->user_id_cvr',
       'user_gender_id->user_occupation_id_cvr',
       'user_gender_id->user_star_level_cvr',
       'user_id->user_occupation_id_cvr', 'user_id->user_star_level_cvr',
       'user_occupation_id->user_star_level_cvr'],
      dtype='object', length=258)

In [50]:
def logloss(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

## train LGBM model

In [51]:
train_train = df_train[df_train.day != 24]
train_valid = df_train[df_train.day == 24]

train_X = train_train.drop('is_trade', axis=1)
train_y = train_train['is_trade'].values

valid_X = train_valid.drop('is_trade', axis=1)
valid_y = train_valid['is_trade'].values

d_train = lgb.Dataset(train_X, label=train_y)
d_valid = lgb.Dataset(valid_X, label=valid_y)
watchlist = [d_train, d_valid]

In [52]:
print('LGBM training')
#lgb_train = lgb.Dataset(df_train.values, label = y_train.is_trade)
params = {
    'task': 'train',
    #'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':5,
#    'num_leaves': 6,
#    'min_data_in_leaf':1,
    #'max_bin':150,  
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction':0.8,
    #'bagging_freq': 5,
    'verbose': 0,
}
model = lgb.train(params, train_set=d_train, num_boost_round=6000, valid_sets=watchlist, \
                  early_stopping_rounds=500, verbose_eval=50)
print('LGBM training done')

LGBM training
Train until valid scores didn't improve in 500 rounds.
[50]	training's binary_logloss: 0.0838663	valid_1's binary_logloss: 0.0824738
[100]	training's binary_logloss: 0.0771171	valid_1's binary_logloss: 0.08245
[150]	training's binary_logloss: 0.0718251	valid_1's binary_logloss: 0.082971
[200]	training's binary_logloss: 0.0670803	valid_1's binary_logloss: 0.0834966
[250]	training's binary_logloss: 0.063006	valid_1's binary_logloss: 0.083919
[300]	training's binary_logloss: 0.0590837	valid_1's binary_logloss: 0.0844388
[350]	training's binary_logloss: 0.0553597	valid_1's binary_logloss: 0.0850025
[400]	training's binary_logloss: 0.0518671	valid_1's binary_logloss: 0.0855254
[450]	training's binary_logloss: 0.048331	valid_1's binary_logloss: 0.0858415
[500]	training's binary_logloss: 0.0453207	valid_1's binary_logloss: 0.0862865
[550]	training's binary_logloss: 0.0426485	valid_1's binary_logloss: 0.0868017
Early stopping, best iteration is:
[71]	training's binary_logloss: 0.

## Add

In [41]:
train_X = train_train.drop(newdropfeature+['is_trade'], axis=1)
train_y = train_train['is_trade'].values

valid_X = train_valid.drop(newdropfeature+['is_trade'], axis=1)
valid_y = train_valid['is_trade'].values

d_train = lgb.Dataset(train_X, label=train_y)
d_valid = lgb.Dataset(valid_X, label=valid_y)
watchlist = [d_train, d_valid]

print('LGBM training')
#lgb_train = lgb.Dataset(df_train.values, label = y_train.is_trade)
params = {
    'task': 'train',
    #'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':4,
#    'num_leaves': 6,
#    'min_data_in_leaf':1,
    #'max_bin':150,  
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction':0.8,
    #'bagging_freq': 5,
    'verbose': 0,
}
model = lgb.train(params, train_set=d_train, num_boost_round=6000, valid_sets=watchlist, \
                  early_stopping_rounds=50, verbose_eval=10)
print('LGBM training done')

LGBM training
Train until valid scores didn't improve in 50 rounds.
[10]	training's binary_logloss: 0.237602	valid_1's binary_logloss: 0.234759
[20]	training's binary_logloss: 0.127523	valid_1's binary_logloss: 0.123409
[30]	training's binary_logloss: 0.0968469	valid_1's binary_logloss: 0.0922758
[40]	training's binary_logloss: 0.0886153	valid_1's binary_logloss: 0.08423
[50]	training's binary_logloss: 0.0860554	valid_1's binary_logloss: 0.0823664
[60]	training's binary_logloss: 0.0848001	valid_1's binary_logloss: 0.0820453
[70]	training's binary_logloss: 0.0837279	valid_1's binary_logloss: 0.0821458
[80]	training's binary_logloss: 0.0827362	valid_1's binary_logloss: 0.082271
[90]	training's binary_logloss: 0.0819797	valid_1's binary_logloss: 0.0824079
[100]	training's binary_logloss: 0.0812107	valid_1's binary_logloss: 0.0825416
[110]	training's binary_logloss: 0.0806159	valid_1's binary_logloss: 0.0825906
Early stopping, best iteration is:
[60]	training's binary_logloss: 0.0848001	va

In [32]:
newdropfeature=[]
for i in range(len(train_X.columns)):
    if model.feature_importance()[i]<=1:
        newdropfeature.append(train_X.columns[i])
    if model.feature_importance()[i]>20:
        print(train_X.columns[i])

shop_score_delivery
hour->item_price_level_cvr
hour->shop_review_num_level_cvr
hour->user_age_level_cvr
hour->user_gender_id_cvr
item_category_hash->item_price_level_cvr
item_collected_level->item_sales_level_cvr
item_pv_level->item_sales_level_cvr
user_age_level->user_star_level_cvr


In [33]:
newdropfeature

['day',
 'item_category_hash',
 'item_city_id',
 'item_collected_level',
 'item_price_level',
 'item_pv_level',
 'item_sales_level',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'user_age_level',
 'user_gender_id',
 'user_occupation_id',
 'hour->user_id_cvr',
 'item_brand_id->item_id_cvr',
 'item_brand_id->user_id_cvr',
 'item_category_hash->item_category_list_cvr',
 'item_category_hash->item_property_list_cvr',
 'item_category_list->item_property_list_cvr',
 'item_category_list->min_cvr',
 'item_category_list->second_cvr',
 'item_category_list->shop_review_num_level_cvr',
 'item_category_list->shop_star_level_cvr',
 'item_category_list->user_occupation_id_cvr',
 'item_city_id->item_id_cvr',
 'item_city_id->item_property_list_cvr',
 'item_city_id->user_id_cvr',
 'item_collected_level->item_id_cvr',
 'item_collected_level->item_property_list_cvr',
 'item_collected_level->user_id_cvr',
 'item_id->item_price_level_cvr',
 'item_id->item_property_list_cvr',
 'item_id->shop_id_cv

In [19]:
model.feature_importance()

array([ 2,  8,  1, 15,  4,  5,  1,  0,  0,  5,  0,  1,  1,  7,  9,  5,  0,
        0, 31, 11, 13,  2,  1,  0,  1,  6, 12,  6, 19,  3, 15,  5,  3,  7,
        4, 10,  5,  9,  4,  3, 10,  4, 18, 11,  6,  7,  4,  8, 16, 12, 10,
        9,  6, 24,  6,  6,  7, 14,  6,  5, 21,  7, 46, 39,  0, 18, 18, 10,
        8, 15,  9,  1, 20,  4, 16, 12,  8,  5,  5,  9,  3, 14,  7,  0,  5,
        5,  0, 18,  5,  2, 26,  1,  5, 20, 13,  8, 17,  6,  4, 10,  4,  5,
        3,  3,  8,  3,  2, 10,  1,  3, 15,  1,  1, 17,  0,  1,  2,  3,  2,
        0,  8,  4,  0, 18,  0, 12,  8,  9,  3,  9,  9, 14,  6, 13,  0,  7,
       17,  0,  4,  0,  4, 36, 10, 14,  6,  3,  5,  7,  4,  0,  5, 12,  0,
        0,  3, 10,  2,  2,  0,  1,  1,  4,  3,  0,  4,  1,  0,  6, 15, 12,
        6, 11,  5, 10, 20,  8,  0,  7, 16,  1,  2,  5,  3,  0,  1,  0,  9,
        3,  0,  7,  1, 23,  6,  5,  3,  3,  3, 11,  3,  0,  5,  9,  4, 10,
        8, 17, 13, 11, 10,  0,  5, 13, 13,  2, 12, 14,  8, 13,  0,  8,  9,
        5,  5,  9, 11,  9

## predict test prob 

In [107]:
instance_id=df_test.instance_id.values 
prob = model.predict(df_test, num_iteration=model.best_iteration )

output=pd.DataFrame({'instance_id':instance_id,'predicted_score':prob})  
output['predicted_score'] = output['predicted_score'].apply(lambda x:round(x,10))
print(output.head())
#output.to_hdf(path + 'output/submission.h5',index=False)
#print('the output path is '+path)

           instance_id  predicted_score
0  2475218615076601065          0.00791
1   398316874173557226          0.01633
2  6586402638209028583          0.01563
3  1040996105851528465          0.01780
4  6316278569655873454          0.01189
